In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import sys

In [22]:
def get_links(result_link):
    """Gets all article links on a given page on a news site"""
    links=[]
    r = requests.get(result_link)
    soup = BeautifulSoup(r.text, 'html.parser')
    links_results = soup.findAll('h1', {"class":"entry-title"}) #Gets article link wrappers
    
    for result in links_results:
        links.append(result.findAll('a')[0].get('href'))

    return links

def get_article(article_link):
    """Takes in a link to an article and returns the formatted article as text"""
    title = ""
    author = ""
    date = ""
    article= ""
    
    r = requests.get(article_link)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    #Getting all relevant info
    title = soup.find('h1', class_='entry-title').text
    author = soup.find('a', {'class': 'author' }).text
    date = soup.find('a', class_='entry-date').text    
    article_body = soup.find(class_='entry-content')

    for p in article_body.findAll('p'):
        excerpt = p.text.replace(u'\xa0', u' ')
        article = article + " " + excerpt
        
    return title, author, date, article

In [23]:
titles = []
authors = []
dates = []
articles=[]
MAX_PAGES = 441
# MAX_PAGES = 2

for i in range(1, MAX_PAGES):    
    links = get_links('https://wattsupwiththat.com/page/{}/'.format(i))
    
    for link in links:
        title, author, date, article = get_article(link)
        
        titles.append(title)    
        authors.append(author)
        dates.append(date)
        articles.append(article) 
        
    sys.stdout.write("\rPage {} done, total articles: {}".format(i, len(articles)))
    sys.stdout.flush()

Page 440 done, total articles: 4405

In [25]:
df = pd.DataFrame(data={'title': titles, 'author': authors, 'date': dates, 'article': articles}).drop_duplicates(keep='first')
df['denial?'] = [1]*len(df.index)
print('Num. articles: {}'.format(len(df)))
df.head(5)

Num. articles: 4404


,title,author,date,article,denial?
0,Aussie Government: UNESCO Should NOT cite Clim...,Eric Worrall,"3 hours ago August 29, 2019",Guest essay by Eric Worrall Adding to the con...,1
1,Nuking Hurricanes,charles the moderator,"7 hours ago August 29, 2019",Reposted From Dr. Roy Spencers Blog August 26...,1
2,"If #HurricaneDorian Hits as a Cat4, Still No L...",Anthony Watts,"9 hours ago August 29, 2019",By Dr. Roy Spencer from his website Atlantic ...,1
3,Prepare to retreat before climate change!,charles the moderator,"11 hours ago August 29, 2019",Reposted from the Fabius Maximus website Larr...,1
4,Morano’s 1-hour interview debunking ‘global wa...,charles the moderator,"15 hours ago August 29, 2019","From TFP Student Action Celebrities, activis...",1


In [26]:
df.to_csv('../../data/right/data_wattsup.csv', index=False)